In [1]:
!pip3 install folium
!pip3 install wget

<class 'OSError'>: "sh" shell not found

In [2]:
import folium
import wget
import pandas as pd

<class 'ModuleNotFoundError'>: No module named 'folium'

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
import requests as rq
from io import BytesIO

url = "https://github.com/GuillermoCharlesSchneider/bees/blob/main/Beekeepers.xlsx?raw=true" #NJ beekeeper excel data
data = rq.get(url).content
df = pd.read_excel(BytesIO(data))
df

In [ ]:
# Select relevant sub-columns: `yard address`, `Lat(Latitude)`, `Long(Longitude)`, `class`
df1 = df[[ 'yardAddress','lat', 'lon']]
df1

In [ ]:
# Start location is 105 Griscom Mill Road
nj_coordinate = [39.305887,-74.758321]
site_map = folium.Map(location=nj_coordinate, zoom_start=10)

In [ ]:
# Create a blue circle at 105 Griscom Mill Road coordinate with a popup label showing its name
circle = folium.Circle(nj_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Griscom Mill Bees'))
# Create a blue circle at 105 Griscom Mill Road coordinate with a icon showing its name
marker = folium.map.Marker(
    nj_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Griscom Mill Bees',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [ ]:
# Initial the map
site_map = folium.Map(location=nj_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, row in df1.iterrows():
    coordinate = [row['lat'], row['lon']]
    folium.Circle(coordinate, radius=100, color='#000000', fill=True).add_child(folium.Popup(row['yardAddress'])).add_to(site_map)
    #folium.map.Marker(coordinate, icon=DivIcon(icon_size=(5,5),icon_anchor=(0,0), html='<div style="font-size: 5; color:#d35400;"><b>%s</b></div>' % row['yardAddress'], )).add_to(site_map)
site_map

In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def assign_marker_color(bee_status):
    if bee_status == 1: 
        return 'green'  #not sick
    else:
        return 'red' #sick
    
df1['marker_color'] = df1['NOT_CREATED_YET_BEE_DISEASE_STATUS'].apply(assign_marker_color)
df1.tail(10)

In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance